In [1]:
import pandas as pd
from dijkstar import Graph, find_path

In [2]:
cells = pd.read_csv('../data/cells.txt', sep=" ", header=None)

In [3]:
cells.rename(columns={0:"id"}, inplace=True)

In [4]:
cells = cells["id"].str.split(":").apply(lambda x: tuple([int(x[0]), int(x[1])])).tolist()

In [5]:
graph = Graph()

for cell in cells:
    poss_neigh = [(cell[0]+i, cell[1]+j) for i in [-1, 0, 1] for j in [-1, 0, 1] ]
    for n in poss_neigh:
        if n in cells:
            graph.add_edge(cell, n, {'cost': 1})

In [6]:
cost_func = lambda u, v, e, prev_e: e['cost']
find_path(graph, (24, 105), (38, 52), cost_func=cost_func)

PathInfo(nodes=[(24, 105), (25, 104), (26, 103), (27, 102), (28, 101), (29, 100), (30, 99), (31, 99), (32, 98), (33, 97), (33, 96), (32, 95), (31, 94), (31, 93), (30, 92), (30, 91), (30, 90), (31, 89), (31, 88), (32, 87), (32, 86), (32, 85), (31, 84), (30, 83), (29, 82), (28, 81), (27, 80), (26, 79), (25, 78), (24, 77), (23, 76), (22, 75), (22, 74), (21, 73), (21, 72), (21, 71), (20, 70), (21, 69), (22, 68), (23, 67), (24, 66), (25, 65), (26, 64), (27, 63), (28, 62), (29, 61), (30, 60), (31, 59), (32, 58), (33, 57), (34, 56), (35, 55), (36, 54), (37, 53), (38, 52)], edges=[{'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'cost': 1}, {'co

## Example to show how to use utils/bfs.py

Contains the following functions:  
`[prepare_graph, save_graph, load_graph, bfs, find_shortest_path, to_tuple, to_cell_id]`

Depending on where you are importing, you may need to do  
`import sys; sys.path.append('..')` if you are in another subdirectory, eg from xuankent/

In [7]:
from utils import bfs

Prepare graph does the same thing as above (returns a Graph object)

In [8]:
graph = bfs.prepare_graph("../data/cells.txt")

Save graph to file

In [9]:
bfs.save_graph(graph, "../data/graph.pkl")

Load saved graph (used without preparing model every single time)

In [10]:
del graph
graph = bfs.load_graph("../data/graph.pkl")

Compute costs to all other reachable cells with BFS (returns a dict)  
This is more efficient and we don't need to call dijkstar.find_path for every other cell.  
Briefly, assuming A -> B -> C, it computes cost from A to C by reusing cost of A to B

In [11]:
start = "24:105"

costs = bfs.bfs(graph, start)

# Just a subset of the dictionary
dict(list(costs.items())[0:15])

{'24:105': 0,
 '23:104': 1,
 '23:105': 1,
 '23:106': 1,
 '24:104': 1,
 '24:106': 1,
 '25:104': 1,
 '25:105': 1,
 '25:106': 1,
 '22:103': 2,
 '22:104': 2,
 '22:105': 2,
 '23:103': 2,
 '24:103': 2,
 '22:106': 2}

This function should be sufficiently fast, or we could add in another optional argument for it to stop computing once we have reached, say 30 cost (30 minutes away from reaching)

In [12]:
%timeit bfs.bfs(graph, start)

17.6 ms ± 2.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Wrapper for dijkstar.find_path (returns just the path to destination cell)  
To be used after we know which is the cell we want to head to

In [13]:
dest = "28:100"

bfs.find_shortest_path(graph, start, dest)

[(24, 105), (24, 104), (25, 103), (26, 102), (27, 101), (28, 100)]

## Trying it out

Assuming we are currently at cell "20:100"

In [14]:
start = "20:100"

graph = bfs.load_graph("../data/graph.pkl")

In [15]:
all_cells = pd.DataFrame({'Location': list(map(bfs.to_cell_id, graph.keys()))})
all_cells.head()

,Location
0,23:104
1,23:105
2,26:79
3,23:106
4,23:107


One **caveat** is that for some of the cells, we might not be able to reach at all

In [16]:
costs = bfs.bfs(graph, start)
find_cost = lambda cell: costs[cell] if cell in costs else None

In [17]:
all_cells['Cost'] = all_cells['Location'].map(find_cost)
all_cells.head()

,Location,Cost
0,23:104,4
1,23:105,5
2,26:79,31
3,23:106,6
4,23:107,7


We can then add in the cost to current time in order to obtain the time for us to reach those cells  

-----**AFTER SOME PREDICTION**-----  

Assuming we want to go to cell "20:120"

In [18]:
dest = "21:120"
path = bfs.find_shortest_path(graph, start, dest)
path

[(20, 100),
 (19, 101),
 (18, 102),
 (17, 103),
 (16, 104),
 (15, 105),
 (14, 106),
 (13, 107),
 (13, 108),
 (14, 109),
 (15, 110),
 (16, 111),
 (17, 112),
 (18, 113),
 (19, 114),
 (20, 115),
 (20, 116),
 (20, 117),
 (20, 118),
 (20, 119),
 (21, 120)]

This is an example when we cannot travel in straight from "20:100" to "21:120".  
So our next move is to head to "19:101" (`path[1]`)  
A visualization of the map and all cells may be required in order to **validate** the results